# 246 2025 Collision Analysis
This notebook is used to precisely determine and graph states of the robot superstructure where collisions would occur. This code does not run on the robot, it's simply a data analysis tool utilising the CAD.

In [62]:
import numpy as np
import trimesh
from trimesh import Trimesh
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import itertools

Go to Onshape > 0000 Full Robot Assembly.

Make sure the elevator is at the lowest position and the pivot is at whatever you want to zero point to be. (I used the handoff position, as it was at week zero.)

Right click the assembly tab > `Export`. `Format`=`GLTF` and `Resoultion`=`Coarse`. Save the file in this folder.

In [2]:
scene = trimesh.load_scene('0000 Full Assembly.gltf')

Also configure these variables if neccessary:

In [ ]:
# Names of nodes in the CAD
axle_node_name = "Dead Axle Tube - 3/4in - 36in (REV-21-2510)"
drive_node_name = "1000 Drive Assembly <1>"
stage0_node_name = "Base <1>"
ramp_node_name = "5000 Ramp <1>"
stage1_node_name = "Stage 1 <1>"
endEffector_node_name = "Coral End Effector Assembly <1>"
stage2_node_name = "Stage 2 <1>"

# Max extension of each stage of the elevator in inches
max_extension_inches = 29

# Number of steps in the simulation
z_resolution = 360
theta_resolution = 360

# (z, theta) in inches and degrees
setpoints = {
    'handoff': (0, 360),
    'L2': (0, 360-(180-5.701485)),
    'L3': (15.750000/2, 360-(180-5.701485)),
    'L4': (53.451516/2, 360-(180+49.298515))
}

Setup code to pull relevant meshes and information out of the model:

In [3]:
# Get the pivot axis of rotation
axle_transform = scene.graph.get(axle_node_name)[0]
pivot_axis = trimesh.transformations.translation_from_matrix(axle_transform)

# Get each moving part as a mesh

# chassis
drive_mesh = scene.subscene(drive_node_name).apply_transform(scene.graph.get(drive_node_name)[0]).to_mesh()
stage0_mesh = scene.subscene(stage0_node_name).apply_transform(scene.graph.get(stage0_node_name)[0]).to_mesh()
ramp_mesh = scene.subscene(ramp_node_name).apply_transform(scene.graph.get(ramp_node_name)[0]).to_mesh()
chassis_mesh = trimesh.util.concatenate([drive_mesh, stage0_mesh, ramp_mesh])

# stage 1
stage1_mesh: Trimesh = scene.subscene(stage1_node_name).apply_transform(scene.graph.get(stage1_node_name)[0]).to_mesh()

# stage 2
endEffector_mesh: Trimesh = scene.subscene(endEffector_node_name).apply_transform(scene.graph.get(endEffector_node_name)[0]).to_mesh()

# for visualization
stage2_mesh: Trimesh = scene.subscene(stage2_node_name).apply_transform(scene.graph.get(stage2_node_name)[0]).to_mesh()

Helper functions for representing robot geometry at a particular state:

In [4]:
def get_transforms_for_state(z, theta):
    stage1_transform = trimesh.transformations.translation_matrix([0, 0, z])
    stage2_transform = trimesh.transformations.translation_matrix([0, 0, 2*z])
    pivot_transform = stage2_transform @ trimesh.transformations.rotation_matrix(np.radians(theta), [1, 0, 0], pivot_axis)
    return ( stage1_transform, stage2_transform, pivot_transform )

def inches_to_meters(inches):
    return inches / 39.3701

# For visualization purposes
def get_scene_for_state(z, theta):
    stage1_transform, stage2_transform, pivot_transform = get_transforms_for_state(z=z, theta=theta)
    s = trimesh.Scene([chassis_mesh])
    s.add_geometry(stage1_mesh, transform=stage1_transform)
    s.add_geometry(stage2_mesh, transform=stage2_transform)
    s.add_geometry(endEffector_mesh, transform=pivot_transform)
    return s

Sanity check: Let's show what the robot should look like at L2

In [ ]:
# Sanity check
get_scene_for_state(z=inches_to_meters(setpoints['L4'][0]), theta=setpoints['L4'][1]).show()
# get_scene_for_state(z=inches_to_meters(0), theta=45).show(viewer='gl')

Set up for the simulation:

In [ ]:
endEffector_collisionManager = trimesh.collision.CollisionManager()
endEffector_collisionManager.add_object("endEffector", endEffector_mesh)

chassis_collisionManager = trimesh.collision.CollisionManager()
chassis_collisionManager.add_object("chassis", chassis_mesh)

stage1_collisionManager = trimesh.collision.CollisionManager()
stage1_collisionManager.add_object("stage1", stage1_mesh)

Now, we can iterate through potential states to check for collisions:

In [6]:
results = []

for z in np.linspace(0, inches_to_meters(max_extension_inches), z_resolution):
    for theta in np.linspace(0, 360, theta_resolution):
        stage1_transform, stage2_transform, pivot_transform = get_transforms_for_state(z=z, theta=theta)
        stage1_collisionManager.set_transform("stage1", stage1_transform)
        endEffector_collisionManager.set_transform("endEffector", pivot_transform)
        chassis_collision = endEffector_collisionManager.in_collision_other(chassis_collisionManager)
        stage1_collision = endEffector_collisionManager.in_collision_other(stage1_collisionManager)
        results.append(dict(z=z, theta=theta, chassis_collision=chassis_collision, stage1_collision=stage1_collision))

results = pd.DataFrame(results)

And plot the results!

In [ ]:
fig = plt.figure()
plt.imshow(results.pivot(columns="z", index="theta", values="chassis_collision"), cmap='Reds', aspect='auto', extent=[0, max_extension_inches, 360, 0])
plt.imshow(results.pivot(columns="z", index="theta", values="stage1_collision"), cmap='Blues', alpha=.5, aspect='auto', extent=[0, max_extension_inches, 360, 0])
plt.xlabel("z -- elevator stage 1 extension (inches)")
plt.ylabel("theta -- pivot angle (degrees)")
fig.legend(handles=[
    mpatches.Patch(color='red', label='Ramp/Stage 0 Collision', alpha=0.5),
    mpatches.Patch(color='blue', label='Stage 1 Collision', alpha=0.5)
])

# Annotate the figure with setpoints
for label, (z, theta) in setpoints.items():
    plt.plot(z, theta, color='black', marker='o')
    plt.annotate(label, (z, theta), textcoords="offset points", xytext=(10,-10))

for pair in itertools.combinations(setpoints.values(), 2): # draw lines between setpoints
    plt.plot(*zip(*pair), color='black', linestyle='--')